## VIPS Benchmarking for 3DHistech Tiff Files

Command structure: 

```
vips tiffsave input_file output_file.tif \
    --compression COMPRESSION \
    --Q QUALITY \
    --pyramid \
    --tile \
    --tile-width TILE_WIDTH \
    --tile-height TILE_HEIGHT
```

Selection of important options:
```bash
Operation
--compression=VipsForeignTiffCompression     Compression for this file (Options are: none, jpeg, deflate, packbits, ccittfax4, lzw, webp, zstd, jp2k)
-Q, --Q=gint                                 Q factor (Value between 0 and 100)
--tile-width=gint                            Tile width in pixels
--tile-height=gint                           Tile height in pixels
--pyramid                                    Write a pyramidal tiff
--properties                                 Write a properties document to IMAGEDESCRIPTION
-d, --depth=VipsForeignDzDepth               Pyramid depth

VIPS Application Options
--vips-concurrency=N                         evaluate with N concurrent threads
--vips-progress                              show progress feedback
--vips-cache-max=N                           cache at most N operations
--vips-cache-max-memory=N                    cache at most N bytes in memory
--vips-cache-max-files=N                     allow at most N open files
```

## Test VIPS for 3D Histech Setup

In [1]:
# imports
from pathlib import Path

# Define Filepath
input_file = "./input_files/3DHistech/Testscans-Fabian/E445_24-1A.1_Big_Jpeg_enhanced/E445_24-1A.1_Wholeslide_EnhancedColors_Extended.tif"
output_folder = "./output_files/VIPS"

# Define Output Path
input_file_path_obj = Path(input_file)
output_folder = Path(output_folder)

output_file = output_folder / input_file_path_obj.parent.name / f"{input_file_path_obj.stem}_vips.tif"
output_file.parent.mkdir(exist_ok=True, parents=True)
output_file = output_file.resolve()

In [ ]:
# tile size test: 256, 512, 1024
# compression: none, jpeg, jp2k
# quality: 100, 95, 90, 80
 

In [5]:
!vips tiffsave $input_file $output_file \
    --pyramid \
    --tile \
    --tile-width 256 \
    --tile-height 256 \
    --vips-progress \
    --vips-concurrency=4 \
    --vips-cache-max-memory=8000000000


(vips:827): VIPS-WARNING **: 14:06:53.759: Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1]

(vips:827): VIPS-WARNING **: 14:06:53.766: Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1]

(vips:827): VIPS-WARNING **: 14:06:53.771: Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1]

(vips:827): VIPS-WARNING **: 14:06:53.775: Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1]

(vips:827): VIPS-WARNING **: 14:06:53.779: Auto-corrected former TIFF subsampling values [2,2] to match subsampling values inside JPEG compressed data [2,1]
vips ./input_files/3DHistech/Testscans-Fabian/E445_24-1A.1_Big_Jpeg_enhanced/E445_24-1A.1_Wholeslide_EnhancedColors_Extended.tif: 67072 x 149760 pixels, 4 threads, 67072 x 1 tiles, 128 lines in buffer

(vips:82

In [ ]:
# todo: analyze runtime, analyze file size, analyze simple pathopatch